# TF-IDF Vectorizer 2-3 ngrams Cosine Similarity
## ORBIS no geographical zone information available matching vs all DENUE

# General information
This notebook mainly executes the following for the firm names in `denue_final` and `orbis_final` that haven't got geographical zone information available: 

- Extracting the company names. 
- Training the algorithm. 
- Extracting the results. 
- Labeling the results. 
- Exporting them to a Comma Separated Values file. 

After this, we've got the possible name matches in `denue_final` for each company in `orbis_final`. 

# Input files
1. **orbis_final:** `'/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/output/orbis_final.csv'` This file contains a data set where each row represents a firm with one of their names associated, also, entity, municipality and ORBIS's BVDID number.
2. **denue_final_alternative:** `'/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/output/denue_final_alternative.csv'` This file contains a dataset where each row represents a firm with one of their names associated, also, the number of workers in that firm, entity, municipality and DENUE's key.

In [1]:
orbis_final_file = '/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/output/orbis_final.csv'
denue_final_alternative_file = '/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/output/denue_final_alternative.csv'

# Output files
1. **final_output:** `'/scratch/public/jpvasquez/MNCs_informality/Final_data/output/2-1-2-4-matches_tf_idf_orbis_no_geo_denue_alternative.csv'` This file contains all possible matches from no geo ORBIS vs all DENUE. 

In [2]:
final_output = '/scratch/public/jpvasquez/MNCs_informality/Final_data/output/2-1-2-4-matches_tf_idf_orbis_no_geo_denue_alternative.csv'

# Packages
These are the needed packages to run this code. In case, the machine you're running this in doesn't have any of these packages, run this code: 

`!pip install package_name`

- **Pandas** is the package which handles importing, wrangling, cleaning and doing everything with the data. 
- **Numpy** is needed in order to declare missing values. 
- **Glob** gets all the files from a directory with a prefix. 
- **Sklearn** is a package for machile learning, we'll use the module for Natural Language Processing. 
- **Scipy** is used for scientific computing, in our case, `csr_matrix` is a dependency of `awesome_cossim_topn`. 
- **Sparse_dot_topn** performs sparse matrix multiplication followed by top-n multiplication result selection. 

In [3]:
import pandas as pd
import numpy as np
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn

# Importing the data

In [4]:
orbis_final = pd.read_csv(orbis_final_file)
denue_final = pd.read_csv(denue_final_alternative_file)

# Declaring options
For customizability's sake, we'll group all the variables, options and arguments we could wish to change in the future. We'll make it in one cell, but feel free to split it into how many cells you want. 

In [5]:
# orbis_final
base_data = orbis_final
category1 = 'entidad'
category2 = 'municipio'
# matching_df
matching_data = denue_final
base_names_variable = 'companyname'
matching_names_variable = 'firm'
# extracting the results
## top results
top_results_n = 100 # number of results of top quality
top_results_lower_bound = 0.95 # lowest score accepted for top quality matches
## uncertain results
uncertain_results_n = 300 # number of results of uncertain quality
uncertain_results_lower_bound = 0.75 # lowest score accepted for uncertain quality matches
uncertain_results_upper_bound = 0.95 # highest score accepted for uncertain quality matches
n_uncertain_results = 5 # top n results for uncertain matches
cores = 55 # subject to the number specified in the batch options

# Filtering the data
Drop the firms that have both categories specified and without a firm name registered. Finally, drop its duplicates. 

In [6]:
base_df = (base_data[(base_data.isna()[category1]) 
                     | (base_data.isna()[category2])] # keep observations that lack one of the specified categories
           .dropna(subset = [base_names_variable]) # drop observations without variable name (firm name)
           .drop_duplicates(ignore_index = True)) # drop possible duplicates
matching_df = (matching_data
               .dropna(subset = [matching_names_variable]) # drop observations without variable name (firm name)
               .drop_duplicates(ignore_index = True)) # drop possible duplicates

# Matching algorithm
The main sections are: 
 
- Get the company names. 
- Train the algorithm. 
- Extract the results. 
    - Top results. 
    - Uncertain results. 
- Save the matches. 

In [7]:
######################################
###Getting variable (company) names###
######################################
base_names = base_df[base_names_variable] # get the list of variable names (not necessarily unique) from base_df
matching_names = matching_df[matching_names_variable] # get the list of variable names (not necessarily unique) from base_df
names = base_names.append(matching_names, ignore_index = True) # concatenating both lists

############################
###Training the algorithm###
############################
vectorizer = TfidfVectorizer(min_df = 1, # call the function, at least one item, 
                             ngram_range = (2, 3), # 2-3 ngrams, 
                             analyzer = 'char') # by characters/letters
tf_idf_matrix = vectorizer.fit(names) # train the models with all the company names from ORBIS and DENUE
tf_idf_matrix_base = tf_idf_matrix.transform(base_names) # transform each observation into a vector
                                                        # and append them into a matrix
tf_idf_matrix_matching = tf_idf_matrix.transform(matching_names) # according to the ngrams
############################
###Extracting the results###
############################
    
############
#Top results
############
possible_matches = awesome_cossim_topn(tf_idf_matrix_base, # sparse matrix multiplication, base_df matrix
                                       tf_idf_matrix_matching.transpose(), # multiplied by the matching_df matrix
                                       top_results_n, top_results_lower_bound, use_threads = True, # options
                                       n_jobs = cores)

possible_matches_base_df_index = possible_matches.nonzero()[0] # positions where the matches are located in base_df
possible_matches_matching_df_index = possible_matches.nonzero()[1] # positions where the matches are located in matching_df
    
certain_matches = (base_df.iloc[possible_matches_base_df_index] # create certain_matches df, merge the firm names of left side
                   .reset_index(drop = True) # select the observations by location, get index to 0, 1, ..., n
                   .merge(matching_df.iloc[possible_matches_matching_df_index] # select the observations by location
                          .reset_index(drop = True), left_index = True, # get index to 0, 1, ..., n
                          right_index = True)) # merge by index, 0 with 0, 1 with 1, ...

certain_matches['score'] = possible_matches.data # assign score to each match
certain_matches['accuracy'] = 'top' # tag them as top results

##################
#Uncertain results
##################
    
possible_matches = awesome_cossim_topn(tf_idf_matrix_base, # sparse matrix multiplication, base_df matrix
                                       tf_idf_matrix_matching.transpose(), # multiplied by the matching_df matrix
                                       uncertain_results_n, uncertain_results_lower_bound, use_threads = True, # options
                                       n_jobs = cores)
    
possible_matches_base_df_index = possible_matches.nonzero()[0] # positions where the matches are located in base_df
possible_matches_matching_df_index = possible_matches.nonzero()[1] # positions where the matches are located in matching_df
    
uncertain_matches = (base_df.iloc[possible_matches_base_df_index] # create certain_matches df, merge the firm names of left side
                     .reset_index(drop = True) # select the observations by location, get index to 0, 1, ..., n
                     .merge(matching_df.iloc[possible_matches_matching_df_index] # select the observations by location
                                .reset_index(drop = True), left_index = True, # get index to 0, 1, ..., n
                            right_index = True)) # merge by index, 0 with 0, 1 with 1, ...
uncertain_matches['score'] = possible_matches.data # assign score to each match
uncertain_matches = (uncertain_matches[uncertain_matches['score'] < uncertain_results_upper_bound] # select the matches below upper bound
                     .sort_values(['score'], ascending = False) # sort values descending
                     .groupby('bvdidnumber').head(n_uncertain_results)) # group by bvdidnumber, then get the n best matches
uncertain_matches['accuracy'] = 'uncertain' # tag them as uncertain results

########################
###Saving the matches###
########################
matches = certain_matches.append(uncertain_matches, ignore_index = True)

# Concatenate and save results
We label the results by algorithm and DENUE's geographical selection, finally, we drop duplicate matches and save it. 

In [8]:
matches['algorithm'] = 'tf-idf' # label the algorithm
matches['selection'] = 'no_geo_alternative' # label the database selection
matches.drop_duplicates(ignore_index = True).to_csv(final_output, index = False)